In [13]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
#Criando uma session com acesso ao gcp
spark = (
    SparkSession
    .builder
    .appName("Desafio-edc-perguntas")
    .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [22]:
df_estabelecimento = spark.read.parquet("gs://bootcamp-edc/refined/estabelecimento.parquet/*.parquet")

In [23]:
dat_columns = ["data_situacao_cadastral","data_inicio_atividade","data_situacao_especial"]
for column in dat_columns:
    df_estabelecimento = df_estabelecimento.withColumn(column,f.to_date(f.col(column),"yyyyMMdd"))

In [24]:
df_estabelecimento.printSchema()

root
 |-- cod_municipio: integer (nullable = true)
 |-- cnpj_basico: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dig_verificador: string (nullable = true)
 |-- idc_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- cod_situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- cod_pais: integer (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal_principal: integer (nullable = true)
 |-- cnae_fiscal_secundaria: integer (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- nome_logradouro: string (nullable = true)
 |-- numero_logradouro: string (nullable = true)
 |-- complemento_logradouro: string (nullable = true)
 |-- nome_bairro: string (nullable = true)
 |-- cod_cep: string (nullable = true)
 |-- sigla_uf: stri

In [25]:
df_estabelecimento.createOrReplaceTempView("tab_estabelecimento")

In [39]:
## Qual o código do CNAE mais presente nas empresas ativas? Quantas empresas utilizam esse CNAE?
spark.sql("""
select cod_atividade, sum(qtd)
from (
select cod_atividade, count(1) as qtd
from tab_estabelecimento
where cod_situacao_cadastral = 2
group by cod_atividade
UNION
select  cod_atividade_secundaria as cod_atividade, count(1) as qtd
from tab_estabelecimento
where cod_situacao_cadastral = 2
group by cod_atividade_secundaria
)
group by cod_atividade
order by sum(qtd) desc
""").show(10)

+-------------+--------+
|cod_atividade|sum(qtd)|
+-------------+--------+
|         null|16111382|
|      4781400| 1115063|
|      9602501|  807286|
|      5611203|  566079|
|      7319002|  512914|
|      4712100|  510644|
|      4399103|  488505|
|      5611201|  410738|
|      5620104|  390550|
|      9602502|  366115|
+-------------+--------+
only showing top 10 rows



In [18]:
## Quantos CNPJs não ativos existem em São Paulo?
spark.sql("""
select sigla_uf, count(1)
from tab_estabelecimento
where cod_situacao_cadastral !=2
group by sigla_uf
order by count(1) desc
""").show(10)

+--------+--------+
|sigla_uf|count(1)|
+--------+--------+
|      SP| 7966472|
|      MG| 3270015|
|      RJ| 2452604|
|      RS| 2229314|
|      PR| 1884566|
|      BA| 1553936|
|      SC| 1224620|
|      GO| 1004579|
|      PE|  947482|
|      CE|  837961|
+--------+--------+
only showing top 10 rows



In [35]:
## Quantas empresas ativas de “Consultoria em tecnologia da informação” existem em Belo Horizonte?
spark.sql(""" Select sigla_uf, desc_municipio, count(1)
from tab_estabelecimento
where cod_situacao_cadastral = 2
AND (lower(desc_atividade) = lower('Consultoria em tecnologia da informação') or
     lower(desc_atividade_secundaria) = lower('Consultoria em tecnologia da informação'))
and lower(desc_municipio) = lower('Belo Horizonte')
group by sigla_uf, desc_municipio
""").toPandas()

,sigla_uf,desc_municipio,count(1)
0,MG,Belo Horizonte,1359


In [28]:
##  Qual o CNAE primário do IGTI? Dica: O IGTI está localizado em Belo Horizonte.
spark.sql(""" Select nome_fantasia, cod_atividade, desc_atividade
from tab_estabelecimento
where upper(nome_fantasia) like '%IGTI%'
and lower(desc_municipio) = lower('Belo Horizonte')
""").toPandas()

,nome_fantasia,cod_atividade,desc_atividade
0,Igti,8532500,Educação superior - graduação e pós-graduação


In [34]:
## Quantas empresas foram abertas desde de 2020?
spark.sql(""" Select count(1)
from tab_estabelecimento
where data_inicio_atividade >= '2020-01-01'
""").toPandas()

,count(1)
0,6314456
